# Quora Insincere Questions Classification
## Detect toxic content to improve online conversations

### Note: Set the colab runtime to None (cpu) for this notebook to work



In [ ]:
!wget https://github.com/ravi-ilango/data-science-ua-2020-nlp/blob/main/lab2/quora_data.zip?raw=true -O quora_data.zip

!unzip quora_data.zip

In [ ]:
import random

import os

from datetime import datetime

#deal with tensors
import torch   

#handling text data
from torchtext import data 

#for attention LSTM
from torch.autograd import Variable


In [ ]:
#Reproducing same results
SEED = 2315

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True 

### Load custom dataset using torchtext.data.TabularDataset

In [ ]:
#loading custom dataset

def tokenizer(text): # create a tokenizer function
    return text.split(' ')

TEXT = data.Field(tokenize=tokenizer,batch_first=True,include_lengths=True)

LABEL = data.LabelField(dtype = torch.float, batch_first=True)

training_data=data.TabularDataset(path = 'quora_data/train.csv',
                                  format = 'csv',
                                  fields = [
                                      (None, None),
                                      ('text', TEXT),
                                      ('label', LABEL)
                                  ],
                                  skip_header = True)

#print preprocessed text
print(vars(training_data.examples[0]))


### Split into training and validation datasets

In [ ]:
train_data, test_data = training_data.split(split_ratio=0.5, random_state = random.seed(SEED))
train_data, valid_data = train_data.split(split_ratio=0.4, random_state = random.seed(SEED))


### Prepare input sequence
This step takes around ~5 min

In [ ]:
#Build vocab dictionary
TEXT.build_vocab(train_data, min_freq=3, vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

In [ ]:
print("Size of TEXT vocabulary: {}\n".format(len(TEXT.vocab)))

print("Size of LABEL vocabulary: {}\n".format(len(LABEL.vocab)))

print("Commonly used words: {}\n".format(TEXT.vocab.freqs.most_common(10)))

#Word dictionary
#TEXT.vocab.stoi

### Prepare training data generators

In [ ]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

### LSTM Classifier

In [ ]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs


#### Instantiate a LSTM Classifier model

In [ ]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)



In [ ]:
#architecture
print(model)

#No. of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding  ()
pretrained_embeddings = TEXT.vocab.vectors
# model.embedding.weight.data.copy_(pretrained_embeddings) # TODO PLEASE USE THIS FOR LSTM

print(pretrained_embeddings.shape)

In [ ]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
device

### Model Train function 

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Model Evaluate function

In [ ]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Check model's forward pass

In [ ]:
#Check model device type
next(model.parameters()).is_cuda, device

In [ ]:
for batch in train_iterator:
    #retrieve text and no. of words
    text, text_lengths = batch.text
    print ("text.shape: ", text.shape)
    print ("text_lengths.shape: ", text_lengths.shape)
    #convert to 1D tensor
    predictions = model(text, text_lengths)
    print ("predictions.shape: ", predictions.shape)
    break

In [ ]:
model_path = 'saved_weights.pt'

### Train the model

This step takes around ~4 min

In [ ]:
N_EPOCHS = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_path)
    
    ts_string = datetime.now().strftime("%m/%d/%Y %H:%M:%S")

    print(f'\n {ts_string} Epoch: {epoch}')
    print(f'\t Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

### Predict

In [ ]:
#load weights
model.load_state_dict(torch.load(model_path));
model.eval();

def prepare_text(sentence):
    # Tokenize
    tokenized = [tok for tok in tokenizer(sentence)]
    # Replace tokens by index from dictionary
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    # Convert to tensors
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1).T 
    length = torch.LongTensor(length)
    return tensor, length

def predict(model, sentence):
    tensor, length = prepare_text(sentence)
    prediction = model(tensor, length)                  #prediction 
    return prediction.item()  

In [ ]:
sentence = "What is your favorite person in history?"
tokenized = [tok for tok in tokenizer(sentence)]
indexed = [TEXT.vocab.stoi[t] for t in tokenized]
[len(indexed)]

In [ ]:
def insincere_or_not(pred):
    return 'Insincere Question' if pred > .5 else 'Normal Question'

In [ ]:
#sincere question
pred = predict(model, "What is your favorite person in history?")
print (insincere_or_not(pred))

In [ ]:
#insincere question
pred = predict(model, "Why Indian girls go crazy about marrying Shri. Rahul Gandhiji?")
print (insincere_or_not(pred))

### Note

This notebook used data and code from a blog in https://www.analyticsvidhya.com